In [6]:
# ==========================================================
# High-Fidelity Artwork Analyzer v6 (Fully Local, Academic Tone)
# ==========================================================
# Features:
# - CIELAB color clustering with region masks
# - Laplacian + Gabor texture fidelity (normalized)
# - FFT-based saliency and subject detection
# - Hough + orientation composition analysis
# - Structural + illumination descriptors
# - Accurate multi-layer overlays (color, texture, saliency, composition)
# - Automatic export of overlay PNGs and curatorial text summary
# ==========================================================

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import os

# ==========================================================
# 1. Utility
# ==========================================================
def load_rgb(image_path):
    img = Image.open(image_path).convert("RGB")
    return np.array(img)

def ensure_dir(path):
    os.makedirs(path, exist_ok=True)

# ==========================================================
# 2. Color Analysis (CIELAB clustering)
# ==========================================================
def analyze_color_lab(img_rgb, k=6):
    H, W = img_rgb.shape[:2]
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    Z = lab.reshape((-1, 3)).astype(np.float32)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, 0.3)
    _, labels, centers = cv2.kmeans(Z, k, None, criteria, 10, cv2.KMEANS_PP_CENTERS)
    labels_2d = labels.reshape(H, W)
    counts = np.bincount(labels.flatten())
    order = np.argsort(-counts)
    centers = centers[order]
    centers_lab = centers.astype(np.uint8).reshape((-1, 1, 3))
    centers_rgb = cv2.cvtColor(centers_lab, cv2.COLOR_Lab2RGB).reshape((-1, 3)).astype(int)
    color_var = float(np.std(centers, axis=0).mean())
    return {"palette_rgb": [tuple(map(int, c)) for c in centers_rgb], "labels": labels_2d, "color_variability": color_var}

# ==========================================================
# 3. Texture Analysis (Laplacian + Gabor)
# ==========================================================
def analyze_texture(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    gray_8u = cv2.normalize(gray, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    lap = cv2.Laplacian(gray_8u, cv2.CV_64F, ksize=3)
    lap_var = float(lap.var())

    # Gabor filters (normalized per-pixel)
    orientations = [0, 30, 60, 90, 120, 150]
    wavelength = max(4, min(gray.shape)//40)
    H, W = gray_8u.shape
    gabor_energies = []
    for theta_deg in orientations:
        theta = np.deg2rad(theta_deg)
        kern = cv2.getGaborKernel((31, 31), 6.0, theta, wavelength, 0.5, 0)
        resp = cv2.filter2D(gray_8u, cv2.CV_32F, kern)
        energy = float(np.mean(resp**2)) / (H * W / 1e5)
        gabor_energies.append(energy)
    gabor_mean = float(np.mean(gabor_energies))
    gabor_anisotropy = float(np.std(gabor_energies))

    return {"laplacian_variance": lap_var, "gabor_mean_energy": gabor_mean, "gabor_anisotropy": gabor_anisotropy}

# ==========================================================
# 4. Saliency (FFT spectral residual)
# ==========================================================
def spectral_residual_saliency(gray_f32):
    eps = 1e-8
    fft = np.fft.fft2(gray_f32)
    log_amp = np.log(np.abs(fft) + eps)
    phase = np.angle(fft)
    avg_log_amp = cv2.blur(log_amp, (3,3))
    spectral_residual = log_amp - avg_log_amp
    saliency = np.abs(np.fft.ifft2(np.exp(spectral_residual + 1j*phase)))**2
    saliency = cv2.GaussianBlur(saliency, (9,9), 2.0)
    sal_norm = (saliency - saliency.min()) / (saliency.max() - saliency.min() + eps)
    return (sal_norm * 255).astype(np.uint8)

def analyze_saliency(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY).astype(np.float32)
    sal = spectral_residual_saliency(gray)
    _, th = cv2.threshold(sal, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    H, W = gray.shape
    if contours:
        x, y, w, h = cv2.boundingRect(max(contours, key=cv2.contourArea))
    else:
        x, y, w, h = W//4, H//4, W//2, H//2
    Y, X = np.indices(gray.shape)
    wts = sal + 1
    cx = int((X*wts).sum() / wts.sum())
    cy = int((Y*wts).sum() / wts.sum())
    return {"saliency_map": sal, "subject_bbox": (x,y,w,h), "focal_point": (cx,cy)}

# ==========================================================
# 5. Composition (edges + Hough)
# ==========================================================
def analyze_composition(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    edges = cv2.Canny(gray, 100, 200)
    linesP = cv2.HoughLinesP(edges, 1, np.pi/180, 120, minLineLength=min(gray.shape)//8, maxLineGap=20)
    lines = []
    if linesP is not None:
        for l in linesP[:,0,:]:
            lines.append(tuple(int(v) for v in l))
    gx = cv2.Sobel(gray, cv2.CV_32F, 1, 0)
    gy = cv2.Sobel(gray, cv2.CV_32F, 0, 1)
    angles = (np.rad2deg(np.arctan2(gy, gx)) + 180) % 180
    mag = np.sqrt(gx**2 + gy**2)
    hist, bins = np.histogram(angles.flatten(), bins=12, range=(0,180), weights=mag.flatten())
    dominant_angle = float(0.5*(bins[np.argmax(hist)] + bins[np.argmax(hist)+1]))
    left_w = edges[:, :gray.shape[1]//2].sum()
    right_w = edges[:, gray.shape[1]//2:].sum()
    balance = (right_w - left_w)/(left_w + right_w + 1e-6)
    return {"edges": edges, "lines": lines, "edge_balance": balance, "dominant_angle_deg": dominant_angle}

# ==========================================================
# 6. Structural & Illumination
# ==========================================================
def analyze_structure(img_rgb):
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    th = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                               cv2.THRESH_BINARY, 25, 5)
    contours, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return {"area_ratio": 0, "aspect_ratio": 1, "eccentricity": 0}
    c = max(contours, key=cv2.contourArea)
    x,y,w,h = cv2.boundingRect(c)
    area_ratio = cv2.contourArea(c)/(gray.shape[0]*gray.shape[1])
    aspect_ratio = w/(h+1e-6)
    ecc = 0
    if len(c) >= 5:
        (cx,cy),(MA,ma),_ = cv2.fitEllipse(c)
        a,b = max(MA,ma)/2, min(MA,ma)/2
        ecc = np.sqrt(max(0, 1-(b*b)/(a*a)))
    return {"area_ratio": area_ratio, "aspect_ratio": aspect_ratio, "eccentricity": ecc}

def analyze_illumination(img_rgb):
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    L = lab[:,:,0].astype(np.float32)
    mu, sigma = L.mean(), L.std() + 1e-6
    skew = (((L - mu)/sigma)**3).mean()
    local = cv2.GaussianBlur(L, (0,0), 3)
    local_var = ((L - local)**2).mean()
    return {"lightness_skew": float(skew), "lightness_local_var": float(local_var)}

# ==========================================================
# 7. Report Generation
# ==========================================================
def academic_report(metrics):
    color = metrics["color"]; texture = metrics["texture"]; comp = metrics["composition"]
    sal = metrics["saliency"]; struct = metrics["structure"]; illum = metrics["illumination"]

    avg = tuple(np.array(color["palette_rgb"]).mean(axis=0).astype(int))
    temp = "warm" if avg[0]>avg[2]+10 else "cool" if avg[2]>avg[0]+10 else "neutral"
    lapv, gmean, ganiso = texture["laplacian_variance"], texture["gabor_mean_energy"], texture["gabor_anisotropy"]
    n_lines, angle, bal = len(comp["lines"]), comp["dominant_angle_deg"], comp["edge_balance"]
    area, aspect, ecc = struct["area_ratio"], struct["aspect_ratio"], struct["eccentricity"]
    skew, lvar = illum["lightness_skew"], illum["lightness_local_var"]
    color_var = color["color_variability"]

    score = (lapv*0.25)+(ganiso*20*0.15)+(n_lines*0.1)+(abs(bal)*40*0.1)+(color_var*0.25)+(abs(skew)*8*0.15)
    if score<18: style="Minimalist"
    elif score<28: style="Realist"
    elif score<40: style="Impressionist"
    elif score<55: style="Expressionist"
    elif score<70: style="Surrealist"
    else: style="Abstract-Contemporary"

    micro = f"{style} tendencies; {temp} palette; structurally {n_lines} linear trajectories; surface variance {lapv:.1f}."
    deep = f"The work demonstrates {style.lower()} qualities with a {temp} chromatic register. Laplacian variance {lapv:.1f} and Gabor anisotropy {ganiso:.2f} indicate surface complexity, while {n_lines} structural lines (balance {bal:.2f}) establish a compositional rhythm around focal point {sal['focal_point']}. Structural form (area ratio {area:.3f}, aspect {aspect:.2f}, ecc {ecc:.2f}) and illumination skew {skew:.2f} complete the formal profile."

    return style, micro, deep

# ==========================================================
# 8. Visualization + Export
# ==========================================================
def visualize_and_export(img_rgb, metrics, out_dir):
    ensure_dir(out_dir)
    sal = metrics["saliency"]["saliency_map"]
    gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    H,W = gray.shape

    # 1. Color map
    lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
    color_overlay = np.zeros_like(img_rgb, dtype=np.float32)
    for c in metrics["color"]["palette_rgb"]:
        rgb_patch = np.uint8([[c]]); lab_patch = cv2.cvtColor(rgb_patch, cv2.COLOR_RGB2LAB)[0][0]
        lower = np.clip(lab_patch - np.array([20,15,15]), 0,255)
        upper = np.clip(lab_patch + np.array([20,15,15]), 0,255)
        mask = cv2.inRange(lab, lower, upper)
        contours,_ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            if cv2.contourArea(cnt) < 0.003*H*W: continue
            cv2.drawContours(color_overlay, [cnt], -1, np.array(c)/255.0, -1)
    cv2.imwrite(os.path.join(out_dir,"overlay_color.png"), (color_overlay*255).astype(np.uint8))

    # 2. Texture heatmap
    lap = np.abs(cv2.Laplacian(gray, cv2.CV_32F))
    lap_norm = cv2.normalize(lap,None,0,1,cv2.NORM_MINMAX)
    thr = np.percentile(lap_norm,98)
    mask = (lap_norm>=thr).astype(np.uint8)
    texture_overlay = np.dstack([mask*255,mask*128,mask*0])
    cv2.imwrite(os.path.join(out_dir,"overlay_texture.png"), texture_overlay)

    # 3. Saliency heatmap
    sal_color = cv2.applyColorMap(cv2.convertScaleAbs(sal), cv2.COLORMAP_JET)
    sal_color = cv2.cvtColor(sal_color, cv2.COLOR_BGR2RGB)
    cv2.imwrite(os.path.join(out_dir,"overlay_saliency.png"), sal_color)

    # 4. Composition lines
    line_overlay = np.zeros_like(img_rgb)
    lines = metrics["composition"]["lines"]
    if lines:
        lengths = [np.hypot(x2-x1,y2-y1) for (x1,y1,x2,y2) in lines]
        cutoff = np.percentile(lengths,90)
        for (x1,y1,x2,y2),L in zip(lines,lengths):
            if L<cutoff: continue
            angle=(np.rad2deg(np.arctan2(y2-y1,x2-x1))+180)%180
            color=(0,255,0) if 60<angle<120 else (255,0,0) if angle<30 or angle>150 else (0,255,255)
            cv2.line(line_overlay,(x1,y1),(x2,y2),color,1,cv2.LINE_AA)
    cv2.imwrite(os.path.join(out_dir,"overlay_composition.png"), line_overlay)

    # 5. Combined composite
    combo = (
        0.5*img_rgb.astype(np.float32)/255 +
        0.3*color_overlay +
        0.2*np.dstack([mask]*3)*np.array([1.0,0.5,0.0]) +
        0.3*sal_color.astype(np.float32)/255 +
        0.5*line_overlay.astype(np.float32)/255
    )
    combo = np.clip(combo,0,1)
    cv2.imwrite(os.path.join(out_dir,"overlay_composite.png"), (combo*255).astype(np.uint8))

# ==========================================================
# 9. Master Routine
# ==========================================================
def analyze_artwork_high_fidelity(image_path):
    img_rgb = load_rgb(image_path)
    metrics = {
        "color": analyze_color_lab(img_rgb),
        "texture": analyze_texture(img_rgb),
        "saliency": analyze_saliency(img_rgb),
        "composition": analyze_composition(img_rgb),
        "structure": analyze_structure(img_rgb),
        "illumination": analyze_illumination(img_rgb)
    }
    style,micro,deep = academic_report(metrics)
    print("\n=== STYLE:",style,"===")
    print("\n— Micro —\n",micro)
    print("\n— Deep —\n",deep)

    out_dir = os.path.splitext(image_path)[0] + "_analysis"
    visualize_and_export(img_rgb,metrics,out_dir)
    with open(os.path.join(out_dir,"curatorial_summary.txt"),"w") as f:
        f.write(f"Style: {style}\n\nMicro:\n{micro}\n\nDeep:\n{deep}")
    print(f"\n✅ Saved overlays and summary in: {out_dir}")

# ==========================================================
# Run
# ==========================================================
if __name__ == "__main__":
    IMAGE_PATH = "/Users/alievanayasso/Documents/SlowMA/mike lot.webp"
    analyze_artwork_high_fidelity(IMAGE_PATH)




=== STYLE: Abstract-Contemporary ===

— Micro —
 Abstract-Contemporary tendencies; warm palette; structurally 150 linear trajectories; surface variance 14760.0.

— Deep —
 The work demonstrates abstract-contemporary qualities with a warm chromatic register. Laplacian variance 14760.0 and Gabor anisotropy 814741.36 indicate surface complexity, while 150 structural lines (balance 1807239253474.24) establish a compositional rhythm around focal point (310, 464). Structural form (area ratio 0.968, aspect 0.80, ecc 0.36) and illumination skew -0.98 complete the formal profile.

✅ Saved overlays and summary in: /Users/alievanayasso/Documents/SlowMA/mike lot_analysis


/var/folders/pq/wbhq49hs45qgf2z43k_2k1vc0000gn/T/ipykernel_8584/3191368765.py:124: RuntimeWarning: overflow encountered in scalar subtract
  balance = (right_w - left_w)/(left_w + right_w + 1e-6)
